In [ ]:
%%html
<style>
h1, h2, h3, h4, h5 {
    color: darkblue;
    font-weight: bold !important;
}
h2 {
    border-bottom: 8px solid darkblue !important;
    padding-bottom: 8px;
}
h3 {
    border-bottom: 2px solid darkblue !important;
    padding-bottom: 6px;
}
.info, .success, .warning, .error {
    border: 1px solid;
    margin: 10px 0px;
    padding:15px 10px;
}
.info {
    color: #00529b;
    background-color: #bde5f8;
}
.success {
    color: #4f8a10;
    background-color: #dff2bf;
}
.warning {
    color: #9f6000;
    background-color: #FEEFB3;
}
.error {
    color: #D8000C;
    background-color: #FFBABA;
}
.language-bash {
    font-weight: 900;
}
.ex {
    font-weight: 900;
    color: rgba(27,27,255,0.87) !important;
}
.mn {
    font-family: Menlo, Consolas, "DejaVu Sans Mono", monospace
}
table {
    margin-left: 0 !important;}
</style>

# 4.6 TextFSM and ntc-templates

-   [TextFSM](https://github.com/google/textfsm) is installed with Netmiko.  It allows one to parse structured text data into Python data types.

-   TextFSM comes with Netmiko.  However, it could be installed via `pip install textfsm`.

<span class='ex'>Example: Parse logfile from network device with TextFSM but a given template</span>

In [ ]:
from netmiko import Netmiko
import requests
import textfsm
import io
import pandas as pd

# In this example, we could use a customized TextFSM template
# to decode data from a "show int" command

# textfsm.TextFSM() expect a file-like object.
# Use io.StringIO to convert string to seekable, iterable file-like object
ret =textfsm.TextFSM(
    io.StringIO(
        requests.get(
            "https://raw.githubusercontent.com/networktocode/ntc-templates/master/templates/cisco_ios_show_interfaces.textfsm"
        ).text
    )
)
# ret.header constains header of data to be parsed

device = Netmiko(
    ip='192.168.88.3',
    username='admin',
    password='cisco',
    device_type='cisco_ios',
    verbose=True,
    secret='class'
)

logdata = device.send_command('show int')
device.disconnect()

data = ret.ParseText(logdata)
df = pd.DataFrame(data, columns=ret.header)
df

## Exercise

Parse returned log data from `show clock` command from a cisco router

In [ ]:
from netmiko import Netmiko
import requests
import textfsm
import io
import pandas as pd

# In this example, we could use a customized TextFSM template
# to decode data from a "show int" command

# textfsm.TextFSM() expect a file-like object.
# Use io.StringIO to convert string to seekable, iterable file-like object
ret =textfsm.TextFSM(
    io.StringIO(
        requests.get(
            "https://raw.githubusercontent.com/networktocode/ntc-templates/master/templates/cisco_ios_show_clock.textfsm"
        ).text
    )
)
# ret.header constains header of data to be parsed

device = Netmiko(
    ip='192.168.88.3',
    username='admin',
    password='cisco',
    device_type='cisco_ios',
    verbose=False,
    secret='class'
)

logdata = device.send_command('show clock')
device.disconnect()

data = ret.ParseText(logdata)
df = pd.DataFrame(data, columns=ret.header)
df

### Solution using TextFSM and ntc-templates

-   Netmiko could use TextFSM to parse returned logfile from `send_command()` automatically.
    This feature is enabled by using ntc-templates library and setting `use_textfsm=True` parameter to `send_command()`.

-   [ntc-templates](https://github.com/networktocode/ntc-templates) contains TextFSM templates to parse structured text data from supported network devices.

-   Netmiko and TextFSM do not install ntc_templates by default.  We could install ntc-templates by

    ```bash
    pip install ntc_templates
    ```

    It will be install in `lib/site-packages/ntc_templates/` under Python's home directory
    
    ```bash
    python -c "import ntc_templates; print(ntc_templates.__path__)"
    ```

-   Netmiko looks for ntc_nvironment variable `NET_TEXTFSM`
    -   `ntc-template/templates/` in user's home directory which could be
        -   `C:\Users\<user_name>\ntc-template/templates/` (for Windows)
        -   `~/ntc-template/templates/templates` (for Linux, Unix and Mac)
    -   I have put up a pull request to allow Netmiko to search the ntc-templates from the standard library before looking the templates from the directory pointed by NET_TEXTFSM environmental variable and then user's home directory.  See https://github.com/ktbyers/netmiko/issues/1520.
    

-   You could create a system environmental variable `NET_TEXTFSM` pointing to that directory.

    -   Windows  
        `setx NET_TEXFSM C:\home\Anaconda3\Lib\site-packages\ntc_templates\templates`
        
    -   Unix-like OS  
        Put the following line in your shell startup script (such as `$HOME/.bashrc`):  
        `export NET_TEXTFSM=/usr/python3/Lib/site-packages/ntc_template/templates`

In [4]:
from netmiko import Netmiko
import pandas as pd

devcfg = {
    'ip'         : '192.168.88.3',    # IP address of the remote switch
    'username'   : 'admin',            # Username to login to the switch
    'password'   : 'cisco',            # Enter password from command line
    'device_type': 'cisco_ios',        # Telnet to a cisco_ios switch
    'secret'     : 'cisco',            # Password that encrypts password
    'verbose'    : True,               # Optional, defaults to False
    'secret'     : 'cisco'             # Password for privilege mode
}

device = Netmiko(**devcfg)
device.enable()  # Enter privileged EXEC mode.

#output = device.send_command('sh run', use_textfsm=True)    # show running-configuration
output = device.send_command('sh int', use_textfsm=True)     # show interface
#output = device.send_command('sh ip int', use_textfsm=True) # show ip interface
#output = device.send_command('sh ip ro', use_textfsm=True ) # show ip route
#output = device.send_command('sh ver', use_textfsm=True   ) # show version

device.disconnect()

df = pd.DataFrame(output)
df

SSH connection established to 192.168.88.3:22
Interactive SSH session established


,interface,link_status,protocol_status,hardware_type,address,bia,description,ip_address,mtu,duplex,...,last_output_hang,queue_strategy,input_rate,output_rate,input_packets,output_packets,input_errors,crc,abort,output_errors
0,GigabitEthernet0/0,up,up (connected),iGbE,5000.0001.0000,5000.0001.0000,,,1500,Full-duplex,...,never,fifo,2000,2000,2241,4455,0,0,,0
1,GigabitEthernet0/1,up,up (connected),iGbE,5000.0001.0001,5000.0001.0001,,,1500,Full-duplex,...,never,fifo,0,0,0,3504,0,0,,0
2,GigabitEthernet0/2,up,up (connected),iGbE,5000.0001.0002,5000.0001.0002,,,1500,Full-duplex,...,never,fifo,0,0,0,3504,0,0,,0
3,GigabitEthernet0/3,up,up (connected),iGbE,5000.0001.0003,5000.0001.0003,,,1500,Full-duplex,...,never,fifo,0,0,0,3505,0,0,,0
4,GigabitEthernet1/0,up,up (connected),iGbE,5000.0001.0004,5000.0001.0004,,,1500,Full-duplex,...,never,fifo,0,0,0,3505,0,0,,0
5,GigabitEthernet1/1,up,up (connected),iGbE,5000.0001.0005,5000.0001.0005,,,1500,Full-duplex,...,never,fifo,0,0,0,3505,0,0,,0
6,GigabitEthernet1/2,up,up (connected),iGbE,5000.0001.0006,5000.0001.0006,,,1500,Full-duplex,...,never,fifo,0,0,0,3504,0,0,,0
7,GigabitEthernet1/3,up,up (connected),iGbE,5000.0001.0007,5000.0001.0007,,,1500,Full-duplex,...,never,fifo,0,0,0,3504,0,0,,0
8,Vlan1,up,up,Ethernet SVI,5000.0001.8001,5000.0001.8001,,192.168.88.3/24,1500,,...,never,fifo,2000,2000,2244,1841,0,0,,


In [ ]:
from netmiko import Netmiko
from getpass import getpass
import datetime as dt

stdout = True
outputfile = './cisco.log'

devcfg = {
    'ip'         : '192.168.88.3',     # IP address of the remote switch
    'username'   : 'admin',            # Username to login to the switch
    'password'   : 'cisco',            # Enter password from command line
    'device_type': 'cisco_ios',        # Telnet to a cisco_ios switch
    'secret'     : 'cisco',            # Password that encrypts password
    'verbose'    : True,               # Optional, defaults to False
}

now = f"{dt.datetime.now()}"[0:19]     # Current date time

device = Netmiko(**devcfg)
device.enable()  # Enter privileged EXEC mode.

output = device.send_command('sh int', use_textfsm=True)     # show interface
device.disconnect()

data = [] # To store a list of ports in a device
for d in output:
    port = {}  # To store data for current port
    port['now']          = now
    port['interface']    = d['interface']
    port['link_status']  = d['link_status']
    port['mtu']          = d['mtu']
    port['input_errors'] = d['input_errors']
    data.append(port)

df = pd.DataFrame(output)
df